In [1]:
GCS_PATH = '/kaggle/input/cassava-leaf-disease-classification'
#../input/cassava-leaf-disease-classification/train_tfrecords

In [2]:
from tensorflow import keras
import functools
_KERAS_BACKEND = None
_KERAS_LAYERS = None
_KERAS_MODELS = None
_KERAS_UTILS = None
def inject_keras_modules(func):
    import keras
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        kwargs['backend'] = keras.backend
        kwargs['layers'] = keras.layers
        kwargs['models'] = keras.models
        kwargs['utils'] = keras.utils
        return func(*args, **kwargs)

    return wrapper
def get_submodules_from_kwargs(kwargs):
    backend = kwargs.get('backend', _KERAS_BACKEND)
    layers = kwargs.get('layers', _KERAS_LAYERS)
    models = kwargs.get('models', _KERAS_MODELS)
    utils = kwargs.get('utils', _KERAS_UTILS)
    for key in kwargs.keys():
        if key not in ['backend', 'layers', 'models', 'utils']:
            raise TypeError('Invalid keyword argument: %s', key)
    return backend, layers, models, utils
def get_vit_submodules_from_kwargs(kwargs):
    #backend = kwargs.get('backend', _KERAS_BACKEND)
    #layers = kwargs.get('layers', _KERAS_LAYERS)
    #models = kwargs.get('models', _KERAS_MODELS)
    #utils = kwargs.get('utils', _KERAS_UTILS)
    num_heads =kwargs.get('num_heads', _KERAS_BACKEND)
    mlp_dim =kwargs.get('mlp_dim', _KERAS_BACKEND)
    dropout =kwargs.get('dropout', _KERAS_BACKEND)
    #for key in kwargs.keys():
    #    if key not in ['backend', 'layers', 'models', 'utils']:
    #        raise TypeError('Invalid keyword argument: %s', key)
    return num_heads,mlp_dim,dropout
def get_swish(**kwargs):
    backend, layers, models, keras_utils = get_submodules_from_kwargs(kwargs)

    def swish(x):
        """Swish activation function: x * sigmoid(x).
        Reference: [Searching for Activation Functions](https://arxiv.org/abs/1710.05941)
        """

        if backend.backend() == 'tensorflow':
            try:
                # The native TF implementation has a more
                # memory-efficient gradient implementation
                return backend.tf.nn.swish(x)
            except AttributeError:
                pass

        return x * backend.sigmoid(x)

    return swish
def get_dropout(**kwargs):
    """Wrapper over custom dropout. Fix problem of ``None`` shape for tf.keras.
    It is not possible to define FixedDropout class as global object,
    because we do not have modules for inheritance at first time.
    Issue:
        https://github.com/tensorflow/tensorflow/issues/30946
    """
    backend, layers, models, keras_utils = get_submodules_from_kwargs(kwargs)

    class FixedDropout(keras.layers.Dropout):
        def _get_noise_shape(self, inputs):
            if self.noise_shape is None:
                return self.noise_shape

            symbolic_shape = backend.shape(inputs)
            noise_shape = [symbolic_shape[axis] if shape is None else shape
                           for axis, shape in enumerate(self.noise_shape)]
            return tuple(noise_shape)

    return FixedDropout
def get_weight_crossentropy(**kwargs):
    backend, layers, models, keras_utils = get_submodules_from_kwargs(kwargs)

    def w_categorical_crossentropy(y_true, y_pred,weights,from_logits=False,label_smoothing=0 ):
        if(weights==None):
            return K.categorical_crossentropy(y_pred, y_true,from_logits,label_smoothing)
        nb_cl = len(weights)
        final_mask = K.zeros_like(y_pred[:, 0])
        y_pred_max = K.max(y_pred, axis=1)
        y_pred_max = K.reshape(y_pred_max, (K.shape(y_pred)[0], 1))
        y_pred_max_mat = K.equal(y_pred, y_pred_max)
        for c_p, c_t in product(range(nb_cl), range(nb_cl)):
              final_mask += (weights[c_t, c_p] * y_pred_max_mat[:, c_p] * y_true[:, c_t])
        return K.categorical_crossentropy(y_pred, y_true,from_logits,label_smoothing) * final_mask

    class WeightCategoricalCrossentropy( keras.losses.CategoricalCrossentropy):
        def __init__(self,
                  from_logits=False,
                  label_smoothing=0,
                  reduction=keras.losses.Reduction.AUTO,
                  name='w_categorical_crossentropy',
                  weights=None):
            @dispatch.add_dispatch_support
            def loss_fun():
                  return  partial(w_categorical_crossentropy, weights= weights)
            super().__init__(
            loss_fun,
            name=name,
            reduction=reduction
            )

    return  WeightCategoricalCrossentropy



import tensorflow as tf
import tensorflow_addons as tfa


class ClassToken(tf.keras.layers.Layer):
    """Append a class token to an input layer."""

    def build(self, input_shape):
        cls_init = tf.zeros_initializer()
        self.hidden_size = input_shape[-1]
        self.cls = tf.Variable(
            name="cls",
            initial_value=cls_init(shape=(1, 1, self.hidden_size), dtype="float32"),
            trainable=True,
        )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        cls_broadcasted = tf.broadcast_to(self.cls, [batch_size, 1, self.hidden_size])
        return tf.concat([cls_broadcasted, inputs], 1)


class AddPositionEmbs(tf.keras.layers.Layer):
    """Adds (optionally learned) positional embeddings to the inputs."""

    def build(self, input_shape):
        assert (
            len(input_shape) == 3
        ), f"Number of dimensions should be 3, got {len(input_shape)}"
        self.pe = tf.Variable(
            name="pos_embedding",
            initial_value=tf.random_normal_initializer(stddev=0.06)(
                shape=(1, input_shape[1], input_shape[2])
            ),
            dtype="float32",
            trainable=True,
        )

    def call(self, inputs):
        return inputs + self.pe


class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, *args, num_heads=16, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_heads = num_heads

    def build(self, input_shape):
        hidden_size =input_shape[-1]
        num_heads = self.num_heads
        if hidden_size % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {hidden_size} should be divisible by number of heads = {num_heads}"
            )
        self.hidden_size = hidden_size
        self.projection_dim = hidden_size // num_heads
        self.query_dense = tf.keras.layers.Dense(hidden_size, name="query")
        self.key_dense = tf.keras.layers.Dense(hidden_size, name="key")
        self.value_dense = tf.keras.layers.Dense(hidden_size, name="value")
        self.combine_heads = tf.keras.layers.Dense(hidden_size, name="out")

    # pylint: disable=no-self-use
    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)

        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.hidden_size))
        output = self.combine_heads(concat_attention)
        return output, weights


class TransformerBlock(tf.keras.layers.Layer):
    """Implements a Transformer block."""

    def __init__(self, *args, num_heads=16, mlp_dim=4096, dropout=0.1, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_heads = num_heads
        self.mlp_dim = mlp_dim
        self.dropout = dropout

    def build(self, input_shape):
        self.att = MultiHeadSelfAttention(
            num_heads=self.num_heads,
            name="MultiHeadDotProductAttention_1",
        )
        self.mlpblock = tf.keras.Sequential(
            [
                tf.keras.layers.Dense(
                    self.mlp_dim, activation=tfa.activations.gelu, name="Dense_0"
                ),
                tf.keras.layers.Dropout(self.dropout),
                tf.keras.layers.Dense(input_shape[-1], name="Dense_1"),
                tf.keras.layers.Dropout(self.dropout),
            ],
            name="MlpBlock_3",
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(
            epsilon=1e-6, name="LayerNorm_0"
        )
        self.layernorm2 = tf.keras.layers.LayerNormalization(
            epsilon=1e-6, name="LayerNorm_2"
        )
        self.dropout = tf.keras.layers.Dropout(self.dropout)

    def call(self, inputs, training):
        x = self.layernorm1(inputs)
        x, weights = self.att(x)
        x = self.dropout(x, training=training)
        x = x + inputs
        y = self.layernorm2(x)
        y = self.mlpblock(y)
        return x + y, weights
custom_objects = {
        'swish': inject_keras_modules(get_swish)(),
        'FixedDropout': inject_keras_modules(get_dropout)(),
        'WeightCategoricalCrossentropy':inject_keras_modules(get_weight_crossentropy)(),
        'ClassToken': ClassToken,
        'AddPositionEmbs': AddPositionEmbs,
        'MultiHeadSelfAttention': MultiHeadSelfAttention,
        'TransformerBlock': TransformerBlock,
}
try:
        keras.utils.generic_utils.get_custom_objects().update(custom_objects)
except AttributeError:
        keras.utils.get_custom_objects().update(custom_objects)


In [3]:
import tensorflow as tf
from tensorflow.keras.layers import *
modelPath3='/kaggle/input/notebook218606b3bc/Vit_512_best.h5'
modelPath4='/kaggle/input/reduve-lr/EffNetB5_512_best (1).h5'
modelPath1='/kaggle/input/1efficient/EffNetB5_512_best (3).h5'
modelPath2="/kaggle/input/densenet2/DenseNet.h5"
model_path_list=[modelPath1,modelPath2,modelPath3,modelPath4]
"""
with tf.device('/gpu:0'):
    model = tf.keras.models.load_model(modelPath2)

"""

"\nwith tf.device('/gpu:0'):\n    model = tf.keras.models.load_model(modelPath2)\n\n"

In [4]:
import math, re, os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from kaggle_datasets import KaggleDatasets
from tensorflow import keras
from functools import partial
from sklearn.model_selection import train_test_split
print("Tensorflow version " + tf.__version__)
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu = None
  gpus = tf.config.experimental.list_logical_devices("GPU")

if tpu:
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu) # Going back and forth between TPU and host is expensive. Better to run 128 batches on the TPU before reporting back.
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
elif len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')
print("Number of accelerators: ", strategy.num_replicas_in_sync)
print('Number of replicas:', strategy.num_replicas_in_sync)
AUTOTUNE = tf.data.experimental.AUTOTUNE
GCS_PATH = '/kaggle/input/cassava-leaf-disease-classification'
BATCH_SIZE = 1#* strategy.num_replicas_in_sync
#BATCH_SIZE=16
VALBATCH_SIZE=BATCH_SIZE
IMAGE_SIZE = [512, 512]
RIMAGE_SIZE=[512//4,512//4]
CLASSES = ['0', '1', '2', '3', '4']
EPOCHS = 10
def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image
def read_tfrecord(example, labeled):
    tfrecord_format = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64)
    } if labeled else {
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_name": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    if labeled:
        label = tf.cast(example['target'], tf.int32)
        return image, label
    idnum = example['image_name']
    return image, idnum
def load_dataset(filenames, labeled=True, ordered=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE)
    return dataset
TRAINING_FILENAMES, VALID_FILENAMES = train_test_split(
    tf.io.gfile.glob(GCS_PATH + '/train_tfrecords/ld_train*.tfrec'),
    test_size=0.35, random_state=5
)
def get_training_dataset(ordered=False):
    dataset = load_dataset(TRAINING_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test_tfrecords/ld_test*.tfrec')

Tensorflow version 2.2.0
Running on single GPU  /device:GPU:0
Number of accelerators:  1
Number of replicas: 1


In [5]:
with tf.device('/gpu:0'):
    TRAINING_FILENAMES, VALID_FILENAMES = train_test_split(
        tf.io.gfile.glob(GCS_PATH + '/train_tfrecords/ld_train*.tfrec'),
        test_size=0.05, random_state=5
    )

    TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test_tfrecords/ld_test*.tfrec')
    def count_data_items(filenames):
        n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
        return np.sum(n)
    NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)

In [6]:
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
        
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape( tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3] )    
    
    # ZOOM MATRIX
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    
    # SHIFT MATRIX
    shift_matrix = tf.reshape( tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3] )
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))
def transform(image,label):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = IMAGE_SIZE[0]
    XDIM = DIM%2 #fix for size 331
    
    rot = 15. * tf.random.normal([1],dtype='float32')
    shr = 5. * tf.random.normal([1],dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    w_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    h_shift = 16. * tf.random.normal([1],dtype='float32') 
    w_shift = 16. * tf.random.normal([1],dtype='float32') 
  
    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
    res=tf.reshape(d,[DIM,DIM,3])
    #res=tf.image.random_brightness(res, 0.2)
    return res,label
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test_tfrecords/ld_test*.tfrec')
def data_augment(image, label):
    # Thanks to the dataset.prefetch(AUTO) statement in the following function this happens essentially for free on TPU. 
    # Data pipeline code is executed on the "CPU" part of the TPU while the TPU itself is computing gradients.
    #image = tf.image.random_flip_left_right(image)
    return transform(image,label)#image, label

In [7]:
def get_test_dataset(ordered=True,tta=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    #dataset = load_dataset(VALID_FILENAMES, labeled=True, ordered=ordered)
    #TRAINING_FILENAMES
    if(tta):
        dataset = dataset.map(data_augment, num_parallel_calls=AUTOTUNE)  
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset
def to_float32(image, label):
    return tf.cast(image, tf.float32), label
#testing_dataset = get_test_dataset()
#test_ds = get_test_dataset(ordered=True) 
#test_ds = test_ds.map(to_float32)
#testing_dataset = get_test_dataset()
#print('Computing predictions...')
#test_images_ds = testing_dataset
#test_images_ds = test_ds.map(lambda image, idnum: image)
#probabilities = model.predict(test_images_ds)
#predictions = np.argmax(probabilities, axis=-1)
#print(predictions)

In [8]:
print(count_data_items(VALID_FILENAMES))

1338


In [9]:
import tensorflow.keras.backend as K
import copy
import tensorflow_addons as tfa
    
with tf.device('/gpu:0'):
    CHANNELS = 3
    N_CLASSES = 5
    TTA_STEPS = 2 # Do TTA if > 0 
    #files_path = f'{database_base_path}/test_images/'
    test_preds = np.zeros((len(os.listdir(GCS_PATH+'/test_images')), N_CLASSES))
    #test_preds = np.zeros((count_data_items(VALID_FILENAMES), N_CLASSES))
    

    for model_path in model_path_list:
        print(model_path)
        K.clear_session()
        model = tf.keras.models.load_model(model_path)
        test_ds = get_test_dataset(tta=False)
        x_test = test_ds.map(lambda image,  idnum: image)
        test_preds += model.predict(x_test) / len((TTA_STEPS+1)*model_path_list)

        if TTA_STEPS > 0:
            test_ds = get_test_dataset( tta=True)
            for step in range(TTA_STEPS):
                print(f'TTA step {step+1}/{TTA_STEPS}')
                x_test = test_ds.map(lambda image,  idnum: image)
                #print(x_test.shape)
                test_preds+= model.predict(x_test) / ((TTA_STEPS+1) * len(model_path_list))
    test_preds = np.argmax(test_preds, axis=-1)
    image_names = [img_name.numpy().decode('utf-8') for img, img_name in iter(test_ds.unbatch())]
    np.savetxt('submission.csv', np.rec.fromarrays([image_names,test_preds]), fmt=['%s', '%d'], delimiter=',', header='image_id,label', comments='')


/kaggle/input/1efficient/EffNetB5_512_best (3).h5
TTA step 1/2
TTA step 2/2
/kaggle/input/densenet2/DenseNet.h5
TTA step 1/2
TTA step 2/2
/kaggle/input/notebook218606b3bc/Vit_512_best.h5


/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/layers/core.py:1030: UserWarning: vit_keras.vit is not loaded, but a Lambda layer uses it. It may cause errors.
  , UserWarning)


TTA step 1/2
TTA step 2/2
/kaggle/input/reduve-lr/EffNetB5_512_best (1).h5
TTA step 1/2
TTA step 2/2


In [10]:
print(test_preds)

[2]
